In [1]:
import requests
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from spectools import lpsd
import scipy.signal.windows as win
from scipy.io import loadmat
import spectools.flattop as ft
from functools import partial

import multiprocessing as mp
ctx = mp.get_context('fork')
pool = ctx.Pool()

In [ ]:
# The following link should lead to a .mat datafile
# Contact the author at spectools@pm.me if the link is broken
url = 'https://www.dropbox.com/s/znmk8tfm0gv8a7b/GFO_Week52_2022.mat?dl=1'

# Download the file
response = requests.get(url)
with open('data.mat', 'wb') as f:
    f.write(response.content)

# Load the .mat file into a NumPy array
data = loadmat('data.mat')
x = np.array(data['RawPhase'][:,0])
print(x)

fs=9.664

In [ ]:
figsize=(8,3)
dpi=150
fontsize=8
linewidth=1.5
xlabel="Sample"
ylabel="Signal"
title="GRACE Follow On LRI range in meters - Calendar week 52, 2022"

fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
ax.plot(x, linewidth=linewidth, label="test", color="gray")
ax.set_xlabel(xlabel, fontsize=fontsize)
ax.set_ylabel(ylabel, fontsize=fontsize)
ax.set_title(title, fontsize=fontsize)
ax.tick_params(labelsize=fontsize)
ax.grid()
fig.tight_layout()
fig.align_ylabels()
plt.show()

In [ ]:
# List of window functions available in scipy.signal.windows
windows_sc = {name: getattr(win, name) for name in dir(win) if callable(getattr(win, name))}
windows_sc

In [ ]:
# List of window functions available in spectools.flattop
windows_ft = {name: getattr(ft, name) for name in dir(ft) if callable(getattr(ft, name))}
windows_ft

In [ ]:
# List of window functions available in scipy.signal.windows
# windows = {name: getattr(win, name) for name in dir(win) if callable(getattr(win, name))}
# windows

windows = {
    'boxcar': win.boxcar,
    'triang': win.triang,
    # 'blackman': win.blackman,
    # 'hamming': win.hamming,
    'hann': win.hann,
    # 'bartlett': win.bartlett,
    # 'flattop': win.flattop,
    # 'parzen': win.parzen,
    # 'bohman': win.bohman,
    # 'blackmanharris': win.blackmanharris,
    'nuttall': partial(win.nuttall, sym=False),
    # 'barthann': win.barthann,
    'kaiser': win.kaiser,
    # 'tukey': win.tukey
    # "SFT3F": ft.SFT3F,
    # "SFT3M": ft.SFT3M,
    # "FTNI": ft.FTNI,
    # "SFT4F": ft.SFT4F,
    # "SFT5F": ft.SFT5F,
    # "SFT4M": ft.SFT4M,
    # "FTHP": ft.FTHP,
    "HFT70": ft.HFT70,
    # "FTSRS": ft.FTSR,
    # "SFT5M": ft.SFT5M,
    # "HFT90D": ft.HFT90D,
    # "HFT95": ft.HFT95,
    # "HFT116D": ft.HFT116D,
    # "HFT144D": ft.HFT144D,
    # "HFT169D": ft.HFT169D,
    # "HFT196D": ft.HFT196D,
    # "HFT223D": ft.HFT223D,
    # "HFT248D": ft.HFT248D
    }
windows

In [8]:
res = {} # Dictionary to store results

In [ ]:
olap = 0.75
Jdes = 1000
Kdes = 100
force_Jdes = True
scheduler='lpsd'

for win_str, win_fun in tqdm(windows.items()):
    res[win_str] = lpsd.ltf(x, fs=fs, win=win_fun, olap=olap, Jdes=Jdes, Kdes=Kdes, adjust_Jdes=force_Jdes, scheduler=scheduler, pool=pool)

In [ ]:
figsize=(6,4)
dpi=150
fontsize=8
linewidth=1.5

fig, ax = plt.subplots(figsize=figsize, dpi=dpi)

for win_str, psd in res.items():
    ax.loglog(psd.f, psd.asd, lw=linewidth, label=win_str)
ax.set_xlim([psd.f[0],psd.f[-1]])
ax.set_xlabel("Fourier frequency (Hz)", fontsize=fontsize)
ax.set_ylabel(r"ASD $\rm (A/Hz^{1/2})$", fontsize=fontsize)
ax.tick_params(axis='both', which='both', labelsize=fontsize)
ax.grid(which='both')
ax.set_title("Comparison of different window functions in a high dynamic range signal", fontsize=fontsize)
ax.legend(loc='best', edgecolor='black', fancybox=True, shadow=True, framealpha=1, fontsize=fontsize, handlelength=2.5)
fig.tight_layout()
plt.show()